In [ ]:
import os
import torch
import torch.nn.functional as F
import pytorch_lightning as pl;
import importlib
import matplotlib.pyplot as plt
import numpy as np
import data
import utils
import sys
import importlib
import wandb
import copy
import pickle

from torch import nn
from torch.nn import functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from abc import ABC,abstractmethod
from torch.utils.data import DataLoader, Subset
from copy import deepcopy
from torch.special import logit
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from timeit import default_timer as timer

importlib.reload(data)
importlib.reload(utils)

pl.seed_everything(42)

debug=False

device=torch.device('mps') if (torch.backends.mps.is_available() and torch.backends.mps.is_built()) else torch.device('cpu')
print(device)


In [ ]:
#testing masked scl

import masking,data,utils,models.MNIST
from data import IRAVENDataModule
importlib.reload(masking)
importlib.reload(data)
importlib.reload(utils)
importlib.reload(models.MNIST)


#setup
train=True
test=False
logging=False
model_ckpt='model_ckpts/MNISTFFNepoch=9-step=9380.ckpt'
n_task=5
n_task_=[1,2,3,4,6,7,8,9]
save_freq= 10000
batch_size=32
device=torch.device('cpu')

#dataset setup
if 1:
    #task dataset
    custom1_datamodule=data.MNISTCustomDataModule(n=n_task)
    custom1_datamodule.setup()
    train_dataloader_task=custom1_datamodule.train_dataloader()
    test_dataloader_task=custom1_datamodule.test_dataloader()

    custom2_datamodule=data.MNISTCustomDataModule(n=n_task_)
    custom2_datamodule.setup()
    test_dataloader_not_task=custom2_datamodule.test_dataloader()

#setup model
if 1:

    model=models.MNIST.MNISTFFN()
    state_dict=torch.load(model_ckpt)['state_dict']
    model.load_state_dict(state_dict)
    model.eval()

#initialise masked model
if 1: 
    init_kwargs={
        'model':model,
        'train_dataloader':train_dataloader_task,
        'test_dataloader1':test_dataloader_task,
        'test_dataloader2':test_dataloader_not_task,
        'device':device,
        'savedir':'model_ckpts/FFN',
    }

    masked_MNISTFFN=masking.MaskedMNISTFFN(init_kwargs)

#train
if 1:
    train_kwargs={
        'alpha':0,
        'lr':1e-3,
        'n_epochs':2 if test else 10,
        'n_batches':20 if test else 'full',
        'val_every_n_steps':5 if test else len(train_dataloader_task),
        'eval_every_n_steps':1e10,
        'n_val_batches':2 if test else 2,
        'n_eval_batches':2 if test else 100,
        'save_freq':save_freq,
        'logging':logging,

        }

    if train:
        masked_MNISTFFN.train(**train_kwargs)
